# wrds_crspm

> Retrieve and process data from WRDS CRSP Monthly Stock File

Since this is a proprietary dataset, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds_crspm

In [ ]:
#|exports
from typing import List
import os

import pandas as pd
import numpy as np
import wrds

import pandasmore as pdm
from finsets import wrds_utils

In [ ]:
#| export
def default_raw_vars():
    """Default variables used in `download` if none are specified. Takes about 2 min to download."""
    return ['permno','permco','date',
            'ret', 'retx', 'shrout', 'prc', 
            'shrcd', 'exchcd','siccd','ticker','cusip','ncusip']            

In [ ]:
print(default_raw_vars())

['permno', 'permco', 'date', 'ret', 'retx', 'shrout', 'prc', 'shrcd', 'exchcd', 'siccd', 'ticker', 'cusip', 'ncusip']


In [ ]:
#| export
def download(vars: List[str]=None, # If None, downloads `default_raw_vars`; else `permno`, `permco`, `date`, and 'exchcd' are added by default
             wrds_username: str=None, #If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
             start_date: str="01/01/1900", # Start date in MM/DD/YYYY format
             end_date: str=None # End date in MM/DD/YYYY format; if None, defaults to current date             
             ) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS crsp.msf and crsp.msenames libraries. 
        Creates `ret_adj` for delisting based on Shumway and Warther (1999) and Johnson and Zhao (2007)"""

    if wrds_username is None:
        wrds_username = os.getenv('WRDS_USERNAME')
        if wrds_username is None: wrds_username = input("Enter your WRDS username: ") 

    if vars is None: vars = default_raw_vars()
    vars = ['permno','permco','date','exchcd'] + [x for x in vars if x not in ['permno','permco','date','exchcd']]

    # Figure out which `vars` come from the `msf` table and which come from the `msenames` table and add a. and b. prefixes
    db = wrds.Connection(wrds_username = wrds_username)
    try:
        all_msf_vars = list(db.describe_table('crsp','msf').name)
        all_mse_vars = list(db.describe_table('crsp','msenames').name)
        my_msf_vars = [f'a.{x}' for x in vars if x in all_msf_vars]
        my_mse_vars = [f'b.{x}' for x in vars if (x in all_mse_vars) and (x not in all_msf_vars)]
        varlist_string = ','.join(my_msf_vars + my_mse_vars)
    except:
        raise RuntimeError("Something went wrong with a WRDS database connection")
    finally: db.close()

    sql_string = f"""SELECT {varlist_string},  c.dlstcd, c.dlret 
                        FROM crsp.msf AS a 
                        LEFT JOIN crsp.msenames AS b
                            ON a.permno=b.permno AND b.namedt<=a.date AND a.date<=b.nameendt
                        LEFT JOIN crsp.msedelist as c
                            ON a.permno=c.permno AND date_trunc('month', a.date) = date_trunc('month', c.dlstdt)                        
                            WHERE a.date BETWEEN '{start_date}' AND COALESCE(%(end)s, CURRENT_DATE) 
                """
    
    df = wrds_utils.download(sql_string, wrds_username=wrds_username, params={'end':end_date})

    #Adjust for delisting returns using Shumway and Warther (1999) and Johnson and Zhao (2007)
    df['npdelist'] = (df['dlstcd']==500) | df['dlstcd'].between(520,584)
    df['dlret'] = np.where(df.dlret.isna() & df.npdelist & df.exchcd.isin([1,2]), -0.35, df.dlret)
    df['dlret'] = np.where(df.dlret.isna() & df.npdelist & df.exchcd.isin([3]), -0.55, df.dlret)
    df['dlret'] = np.where(df.dlret.notna() & df.dlret < -1, -1, df.dlret)
    df['dlret'] = df.dlret.fillna(0)

    df['ret_adj'] = (1 + df.ret) * (1 + df.dlret) - 1
    df['ret_adj'] = np.where(df.ret_adj.isna() & (df.dlret!=0), df.dlret, df.ret_adj)

    return df 

In [ ]:
#| eval: false
raw = download(start_date='01/01/2021')

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done


In [ ]:
#| eval: false
raw.head()

,permno,permco,date,ret,retx,shrout,prc,cusip,exchcd,shrcd,siccd,ticker,ncusip,dlstcd,dlret,npdelist,ret_adj
0,10026.0,7976.0,2021-01-29,-0.017442,-0.017442,18980.0,152.660004,46603210,3.0,11.0,2052.0,JJSF,46603210,NaN,0.0,False,-0.017442
1,10028.0,7978.0,2021-01-29,0.203846,0.203846,26925.0,6.260000,29402E10,2.0,11.0,5094.0,ELA,29402E10,NaN,0.0,False,0.203846
2,10032.0,7980.0,2021-01-29,-0.016494,-0.016494,28766.0,76.919998,72913210,3.0,11.0,3670.0,PLXS,72913210,NaN,0.0,False,-0.016494
3,10044.0,7992.0,2021-01-29,0.071605,0.071605,6074.0,4.340000,77467X10,3.0,11.0,2060.0,RMCF,77467X10,NaN,0.0,False,0.071605
4,10051.0,7999.0,2021-01-29,-0.068213,-0.068213,38109.0,20.490000,41043F20,1.0,11.0,4813.0,HNGR,41043F20,NaN,0.0,False,-0.068213


In [ ]:
#| export
def clean(df: pd.DataFrame=None, # If None, downloads `vars` using `download` function; else, must contain `permno` and `date` columns
          vars: List[str]=None, # If None, downloads `default_raw_vars`
          wrds_username: str=None, #If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
          start_date: str="01/01/1900", # Start date in MM/DD/YYYY format
          end_date: str=None, # End date. Default is current date          
          clean_kwargs: dict={}, # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
          ) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`. If `df` is None, downloads `vars` using `download` function."""

    if df is None: df = download(vars=vars, wrds_username=wrds_username, start_date=start_date, end_date=end_date)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='date', freq='M', **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df = clean(df=raw)

In [ ]:
#| eval: false
df.head(0)

,,date,dtdate,permco,ret,retx,shrout,prc,cusip,exchcd,shrcd,siccd,ticker,ncusip,dlstcd,dlret,npdelist,ret_adj
permno,Mdate,,,,,,,,,,,,,,,,,


In [ ]:
#| eval: false
df = clean(vars=['ret','shrcd','exchcd'], start_date='01/01/2020', end_date='12/31/2020')

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done


In [ ]:
#| eval: false
df.head(0)

,,date,dtdate,permco,ret,exchcd,shrcd,dlstcd,dlret,npdelist,ret_adj
permno,Mdate,,,,,,,,,,


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()